## Apprivoisez les Pandas et leurs _DataFrames_

### Généralités

Dr. X est un grand habitué des tableurs, comme Origin, Excel et Calc. Le format tableau permet de visualiser efficacement les données, de les manipuler et d'avoir une grande flexibilité sur leur contenu (texte, nombres, dates). Dr. X ne se perd pas dans ses tableaux grâce aux en-têtes des colonnes, et peut les classer, les filtrer selon ses goûts. Dr. X apprécie la possibilités de tracer des graphes à la volée, même si ceux-ci ne sont pas tout à fait de qualité _publication_. Dr. X est également heureux de savoir que ses fichiers sont rapidement exportables en un format texte, de type `.csv` ou  `.tsv`.

Dr. X sera donc ravi d'en apprendre plus sur les `Pandas.Dataframe`, un type d'objet qui vont s'utiliser comme des tableurs en Python. Nous verrons dans ce tutorial comment comment construire ou importer et exporter un `DataFrame`, puis comment manipuler les données au sein de ces `DataFrame`. L'objectif va être de 'retrouver les sensations d'Excel' le plus rapidement possible. 

### Créer un DataFrame

Il est possible de créer un `DataFrame` _ex nihilo_ à partir d'un dictionnaire Python, mais nous verrons rapidement que cela n'est probablement pas la manière la plus efficace de les générer. Nous verrons rapidement ensuite comment les importer, par exemple depuis un fichier `.csv`. 

#### À partir d'un dictionnaire ou d'un tableau NumPy

Reprenons notre inénarrable exemple de notre groupe de TD et de ses notes. Cela confine à l'obsession, mais c'est pour la bonne cause. Créons un dictionnaire contenant quatre clés : 

* Prénom
* Nom
* Note TD1
* Note TD2

Compte tenu de la manière dont on a créé notre dictionnaire, il va être assez difficile, par exemple, d'obtenir la note de `Debasish` au TD2. Il faudrait, pour cela, retrouver l'_indice_ auquel on trouve `Debasish` dans la `list` correspondant à `first_name` et examiner la note de l'indice correspondant dans `TD_2`. 

Il existe une fonction [numpy.where()](https://numpy.org/doc/stable/reference/generated/numpy.where.html) qui renvoie un tel indice à partir d'un objet `np.ndarray`, mais vous pouvez constater que cette solution n'est pas très intuitive.

In [ ]:
import numpy as np

my_dict = {'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[18,14,13,11,16,12],
            'TD_2':[13,17,15,13,10,9],
            }

### Numpy solution
# Find index where `Debasish` is found in `first_names`
# Note : np.where returns two arguments, one being empty if we are looking in 1D arrays
idx_match,  = np.where(np.array(my_dict['first_name']) == 'Debasish') 
grade = my_dict['TD_2'][idx_match[0]] # 
print('Grade is : ' + str(grade))

C'est ici que les `DataFrame` vont nous aider ! On peut les créer directement à partir d'un dictionnaire, d'un tableau `np.ndarray`, ou bien d'autres types de données. La plupart du temps, on peut simplement en créer un en invoquant la commande `pd.DataFrame()` et en mettant en entrée les données que vous voulez voir converties. Regardons ce que ça donne si on essaie avec un dictionnaire : 

In [ ]:
import pandas as pd             # Not the best shorthand for French people ...

my_df = pd.DataFrame(my_dict)
my_df

Vous voyez ici déjà que l'affichage de vos données est plutôt joli par rapport à ce que Python vous sert en général, et ressemble assez furieusement à un tableau Excel. Chouette, non ? Le format a également l'air de bien s'accommoder de mélanger des chaînes de caractère et des chiffres, mais ça, c'était déjà le cas avec les listes et bien d'autres types de variables en Python.

__Exercice__ : Essayez de faire la même chose à partir d'un tableau `np.ndarray`, et regardez comment les données s'organisent. 

#### À partir d'un fichier .csv, voire Excel

La meilleure manière de créer des `pd.DataFrame` reste cependant des les importer depuis un fichier formatté avec la bien-nommée fonction `pd.read_csv()`. Essayons de lire mon fichier de rhéologie préféré, `rheology.csv`, qui a le mauvais goût d'être formatté à la _française_, c'est à dire avec des `;` comme séparateurs entre colonnes et des `,` comme séparateurs des décimales. 

In [ ]:
my_rheol = pd.read_csv('files/rheology.csv') # 'What could go wrong ?'
my_rheol                                     # Really wrong things

C'est le bazar complet, mais ne vous inquiétez pas, il est possible de spécifier à la fois : 

* Le délimiteur des champs, `sep` ou `delimiter`
* Le séparateur des décimales, `decimal`
* Un nombre de lignes à ignorer au début du fichier, `skiprows`

Il existe [bien d'autres options](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) si vous avez encore des problèmes pour des types de fichiers ou des manières de les lire bien particuilers.

In [ ]:
my_rheol = pd.read_csv('files/rheology.csv', decimal=',', sep=';')
my_rheol

Le résultat est bien plus satisfaisant ! 

Si vous ne voulez _vraiment_ pas vous embêter avec ces paramètres régionaux, vous pouvez également simplement importer un fichier _Excel_ avec `pd.read_excel()`. Celui-ci 'comprend' les fichiers : 
* `.xls`, par l'intermédiaire du module `xlrd`
* `.xlsx`, par l'intermédiaire du module `openpyxl`
* `.ods`, par l'intermédiaire du module `odfpy` 

Dans notre espace de travail, seul `openpyxl` est installé, il ne vous sera donc pas possible d'importer d'autres types de fichiers que `.xslx`. 

In [ ]:
my_data = pd.read_excel('files/rheology.xlsx')
my_data

### Accéder aux données d'un DataFrame

Le format tableur est bien joli, mais il serait quand même chouette de pouvoir récupérer les notes de notre élève de manière efficace, non ? Voire de modifier les valeurs du tableau en cas d'erreur. 

#### Indexation standard (`[]`)

Pour accéder à une colonne de notre tableau, on peut travailler comme avec les dictionnaires, ou comme avec des objets. Accédons aux prénoms de notre `pd.DataFrame` :

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[18,14,13,11,16,12],
            'TD_2':[13,17,15,13,10,9]})

my_df['first_name']   # Does work 
# my_df.first_name        # Would also have worked fine, unless column has a weird name, e.g. '1' 

Ce que renvoie notre commande `my_df['first_name']` ressemble à un tableau Numpy `np.ndarray` ou une liste, mais ce n'est pas tout à fait le cas, car à gauche vous voyez une liste d'indices. C'est parce que cet objet est en fait une `Series` Pandas, la version '1D' des `DataFrame`. On peut d'ailleurs les convertir très simplement en tableaux Numpy. 

On peut assez facilement sélectionner deux ou plusieurs colonnes parmi toute notre série de données. Pour cela, on remplace notre nom de colonne (par exemple, `first_name'`) par une liste (par ex., [`first_name`, `last_name`]) et le tour est joué ! 

In [ ]:
my_names = my_df[['first_name', 'last_name']]   # Don't forget the extra square brackets !
print(my_names)

Comme pour les `np.ndarray` et les listes, on peut effectuer des coupes au niveau des colonnes avec la syntaxe `[a:b:step]`, comme par exemple ici si je ne veux regarder qu'un étudiant sur deux (même si cela est assez curieux comme pratique) : 

In [ ]:
my_df[::2]

La puissance des `pd.DataFrame` vient du fait que l'on peut __cumuler  facilement les sélections sur les lignes et les colonnes__. On peut tout à fait effectuer la chose suivante : 

In [ ]:
my_df['first_name'][::2]

#### Indexation booléenne 

Enfin, comme pour les [`np.ndarray`](./Application_A_Numpy.ipynb#Indexation-logique), on peut construire des _conditions_ sur construire des sur les valeurs prises dans les cases de notre tableau et effectuer des _coupes_ à partir de ces conditions. _Hein ?_ Voyez plutôt, si je veux récupérer la note de Serguey au TD1. Je vais commencer par trouver la _ligne_ correspondant à Serguey dans la liste des prénoms. Fort pratique ! 

In [ ]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})
            
is_serguey = my_df['first_name'] == 'Serguey'
my_grade = my_df[is_serguey]['TD_1']
my_grade

Je peux même demander à appliquer _plusieurs_ conditions booléennes. Essayons par exemple de trouver un.e éventuel.le élève qui a moins que la moyenne aux deux TD, un élève en _difficulté_, même si ici les résultats sont plutôt bons. On utilisera alors [les opérateurs `~`, `&`, `|` et `^`](./Application_A_Numpy.ipynb#Combinaisons-de-conditions-sous-Numpy), et des fonctions Numpy, qui s'appliquent tout à fait correctement sur les colonnes de mon `pd.Dataframe`. 

In [ ]:
belowavg_td1 = my_df['TD_1'] < np.mean(my_df['TD_1'])       # Values below average of all values for TD1
belowavg_td2 = my_df['TD_2'] < np.mean(my_df['TD_2'])       # Values below average of all values for TD2
my_df[belowavg_td1 & belowavg_td2]

__Exercice__ :

* Pour le fichier de rhéologie `'files/rheology.csv'`, essayez de tracer avec [Matplotlib](./Application_B_Matplotlib.ipynb) la courbe contrainte ('Shearstress') vs. taux de déformation ('Shearrate') correspondant à l'étape 'Flow Curve Down Forward' (contenue dans la colonne 'Name').

In [ ]:
# Pandas and Matplotlib ... Fuuuuuuussssiiiioooonnnn ! 

#### Encore plus de flexibilité : l'indexation avec `.loc[]`

Pandas recommande d'utiliser cette fonction afin d'indexer les `pd.DataFrame`. Hormis le fait que cette 'fonction' soit suivie de crochets `[` et `]` et non de parenthèses `(` et `)`, celle-ci fonctionne d'une manière très similaire à l'indexation simple et booléenne. La syntaxe est en fait encore un peu plus flexible  et permet de combiner plusieurs conditions (par exemple, un nom de colonne et une sélection sur les lignes) entre les mêmes crochets, ce qui n'est pas possible avec l'indexation simple `[]`.


In [89]:
import pandas as pd

my_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]})

is_serguey = my_df['first_name'] == 'Serguey'
my_grade_loc = my_df.loc[is_serguey, 'TD_1']    # Combining two conditions in bet
# my_grade_old = my_df[is_serguey, 'TD_1']        # This does not work

print(my_grade_loc)

1    14
Name: TD_1, dtype: int64


Nous verrons plus loin que les indexations avec `[]` et avec `.loc[]` fonctionnent _différemment lorsqu'on essaie de modifier des valeurs_ du `pd.DataFrame`.

#### Sélectionner une ligne à la main avec `.iloc[]`

Parfois, il vous sera nécessaire de sélectionner _à la main_ une ligne de votre tableau indépendamment de toute condition. Par défaut, les lignes des `DataFrames` sont indexées par des entiers, ce qui permet d'écrire :

```
    my_df[0]
```
pour demander à accéder à la première ligne. 

Mais parfois, vos DataFrames auront des indices qui ne sont pas dans l'ordre, _voire_ des indices qui ne sont pas des chiffres ! Voyons un exemple pour des lignes qui ne sont pas _indexées dans l'ordre_, et essayons naïvement de demander la première ligne du tableau à l'aide de l'indexation normale `[]` puis avec `.loc[]` : 

In [114]:
import pandas as pd

my_sneaky_df = pd.DataFrame({'first_name':['Olivier', 'Serguey', 'Aika', 'Debasish', 'Muhammad', 'Lu' ],
            'last_name': ['Margoulin', 'Prigogine', 'Tokogawa', 'Sarker', 'Ben Amar', 'Ai'],
            'TD_1':[11,14,13,15,12,15],
            'TD_2':[13,17,15,14,16,9]}, index=[5,0,3,2,1,4])

# my_df[0]            # Fails miserably
my_sneaky_df.loc[0]        # Not what I want

first_name      Serguey
last_name     Prigogine
TD_1                 14
TD_2                 17
Name: 0, dtype: object

Aïe ! L'indexation normale `[]` renvoie carrément une erreur, tandis que la version avec `.loc[]` renvoie naturellement la ligne correspondant à l'indice 0, mais qui n'est plus la première ligne de mon tableau. 

C'est pour ce genre de cas que la fonction `.iloc[]` existe. On peut lui préciser un indice seul, mais également des _coupes_ du type `[start:stop:step]` et des listes d'indices. Commentez et dé-commentez les lignes suivantes pour examiner le fonctionnement de `.iloc[]`. 

In [121]:
my_sneaky_df.iloc[0]
# my_sneaky_df.iloc[::3] # Also works 
# my_sneaky_df.iloc[[0,2,4]] # Works too ! Yay !

first_name      Olivier
last_name     Margoulin
TD_1                 11
TD_2                 13
Name: 5, dtype: object